In [ ]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   1%|          | 606k/84.1M [00:00<00:14, 5.92MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:00<00:00, 90.5MB/s]


In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [ ]:
print(vars(train_data.examples[0]))

{'text': ['Before', 'Cujo', ',', 'there', 'was', 'Lucky', 'the', 'devil', 'dog', '.', 'In', '1978,on', 'Halloween', 'night', 'the', 'movie"Devil', 'Dog', ',', 'The', 'Hound', 'of', 'Hell', '"', 'premiered', '.', 'A', 'story', 'of', 'a', 'family', 'getting', 'a', 'new', 'puppy', '(', 'from', 'a', 'farmer', 'who', 'just', 'happen', 'to', 'be', 'in', 'the', 'neighborhood', 'selling', 'fruits', 'and', 'vegetables', ')', 'because', 'their', 'dog', 'Skipper', 'was', 'killed', '.', 'Coencidence', '?', 'Everyone', 'loves', 'the', 'new', 'dog', ',', 'but', 'there', 'is', 'something', 'strange', 'about', 'him', '.', '<', 'br', '/><br', '/>It', 'is', "n't", 'long', 'until', 'the', 'father', 'Mike', 'Barry(Richard', 'Crenna', ',', 'First', 'Blood)starts', 'to', 'notice', '.', 'His', 'wife', 'Betty(Yvette', 'Mimieux', ',', 'Where', 'The', 'Boys', 'Are', ',', 'Jackson', 'County', 'Jail', ',', 'Snowbeast)is', 'different', 'and', 'his', 'kids', 'Charlie', 'and', 'Bonnie(Ike', 'Eisenman', ',', 'Witch',

In [ ]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203063), (',', 192343), ('.', 166129), ('and', 109794), ('a', 109526), ('of', 100808), ('to', 93971), ('is', 76428), ('in', 61581), ('I', 54306), ('it', 53609), ('that', 49177), ('"', 44610), ("'s", 43276), ('this', 42373), ('-', 36806), ('/><br', 35659), ('was', 35096), ('as', 30590), ('with', 30113)]


In [ ]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [ ]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
class SentimentRNN(nn.Module):
    def __init__(self,no_layers,vocab_size,hidden_dim,embedding_dim,drop_prob=0.5):
        super(SentimentRNN,self).__init__()
 
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
 
        self.no_layers = no_layers
        self.vocab_size = vocab_size
    
        # embedding and LSTM layers

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm
        self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
                           num_layers=no_layers, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
    
        # linear and sigmoid layer
        self.fc = nn.Linear(self.hidden_dim, output_dim)
        self.sig = nn.Sigmoid()
        
    def forward(self,x,hidden):
        batch_size = x.size(0)
        # embeddings and lstm_out
        embeds = self.embedding(x)  # shape: B x S x Feature   since batch = True
        #print(embeds.shape)  #[50, 500, 1000]
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)

        sig_out = sig_out[:, -1] # get last batch of labels

         # return last sigmoid output and hidden state
        return sig_out, hidden
        
        
        
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        h0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)

        c0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden

        


In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [ ]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 30

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 49.84%
	 Val. Loss: 0.694 |  Val. Acc: 49.36%
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.53%
	 Val. Loss: 0.695 |  Val. Acc: 49.56%
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 0.694 | Train Acc: 50.04%
	 Val. Loss: 0.694 |  Val. Acc: 49.44%
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.16%
	 Val. Loss: 0.694 |  Val. Acc: 49.51%
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.03%
	 Val. Loss: 0.694 |  Val. Acc: 49.70%
Epoch: 06 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.46%
	 Val. Loss: 0.695 |  Val. Acc: 49.84%
Epoch: 07 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.12%
	 Val. Loss: 0.695 |  Val. Acc: 49.75%
Epoch: 08 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.00%
	 Val. Loss: 0.694 |  Val. Acc: 49.71%
Epoch: 09 | Epoch Time: 0m 14s
	Train Loss: 0.693 | Train Acc: 50.36%
	 Val. Loss: 0.694 |  Val. Acc: 49.71%
Epoch: 10 | Epoch T